In [1]:
# LongWriter - Writing 10,000 Words In Just One Generation
# Sam Witteveen
# https://www.youtube.com/watch?v=6cubCIupyik

# LongWriter GtHub
# https://github.com/THUDM/LongWriter

# LongWriter Paper
# https://arxiv.org/abs/2408.07055

In [2]:
# Use T4 x2 GPU

In [3]:
!pip -q install accelerate
#!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q sentencepiece tokenizer
!pip -q install tiktoken

In [4]:
!pip show transformers

Name: transformers
Version: 4.44.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: kaggle-environments


In [5]:
device = "cuda"

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#model_name = "THUDM/LongWriter-llama3.1-8b"
model_name = "THUDM/LongWriter-glm4-9b"

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             trust_remote_code=True,
                                             device_map="auto")
model = model.eval()



tokenizer_config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/LongWriter-glm4-9b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/4.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

In [7]:
query = "Write a 3000-word story about a poor engineering student who creates AGI and takes over the world. Write in the style of John Grisham."

prompt = f"[INST]{query}[/INST]"

input = tokenizer(prompt, truncation=False,
                  return_tensors="pt").to(device)


# Create a text streamer
from transformers import TextStreamer
streamer = TextStreamer(
    tokenizer,
    skip_prompt=True, # If you set to false, the model will first return the prompt and then the generated text
    skip_special_tokens=True
)


context_length = input.input_ids.shape[-1]
output = model.generate(
    **input,
    streamer=streamer,
    max_new_tokens=32768,
    num_beams=1,
    do_sample=True,
    temperature=0.5,
)[0]

response = tokenizer.decode(output[context_length:], skip_special_tokens=True)
#print(response)

### The Background

Jameson Carter was born into a family that struggled against the relentless tide of poverty. Raised in a dilapidated apartment in the rough outskirts of Detroit, his childhood was marked by scarcity and hardship. His father, a construction worker, was often out of a job, and his mother, a single mother working two jobs, barely managed to keep food on the table. Despite these challenges, Jameson's intellectual curiosity and insatiable thirst for knowledge never waned. His parents, though financially strained, always encouraged his education, believing that education was the key to escaping their fate.

From an early age, Jameson showed a remarkable aptitude for science and mathematics. He would spend hours buried in books, his mind racing with ideas and theories. His teachers noticed his potential and did their best to nurture it, often going out of their way to provide him with extra resources and support. Despite the odds, Jameson managed to excel academically, ear

In [8]:
# Write the response text to a txt file

# Open a file in write mode ('w')
with open('story.txt', 'w') as file:
    # Write the text variable to the file
    file.write(response)


In [9]:
!ls

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


__notebook__.ipynb  story.txt
